In [1]:
from datasets import load_dataset

/home/bt19d200/NER_Vamshi/NER_Model/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset=load_dataset("DFKI-SLT/conll04")

In [3]:
entity_labels=set()
for i in range(len(dataset['test'])):
    for j in dataset['test'][i]['entities']:
        entity_labels.add(j['type'])

In [4]:
entity_labels

{'Loc', 'Org', 'Other', 'Peop'}

In [5]:
test=dataset['test']

In [6]:
test[0]['entities']

[{'end': 7, 'start': 5, 'type': 'Org'},
 {'end': 9, 'start': 8, 'type': 'Other'},
 {'end': 11, 'start': 10, 'type': 'Loc'},
 {'end': 18, 'start': 17, 'type': 'Other'}]

In [7]:
entity_labels_count_true={}
for i in range(len(dataset['test'])):
    for j in dataset['test'][i]['entities']:
        if j['type'] not in entity_labels_count_true:
            entity_labels_count_true[j['type']]=0
        entity_labels_count_true[j['type']]+=1

In [8]:
ner=[]
for i in range(len(test)):
    temp=[]
    for j in test[i]['entities']:
        text=' '.join(test[i]['tokens'][j['start']:j['end']])
        temp.append([j['start'], j['end']-1,j['type'],text])
    ner.append(temp)

In [9]:
entity_other={}
for i in range(len(ner)):
    for j in ner[i]:
        if j[2]=='Other':
            if j[3] not in entity_other:
                entity_other[j[3]]=0
            entity_other[j[3]]+=1
        

In [10]:
entity_other

{'Arab': 1,
 'Palestinians': 1,
 'American': 2,
 'Dec. 13 ,': 1,
 'Dec. 16 ,': 1,
 '1973': 1,
 '1974': 1,
 'Watergate': 2,
 'Cuban missile crisis': 1,
 'Armenian': 1,
 'British': 1,
 '100 million years': 1,
 '69 mph': 1,
 '57 mph': 1,
 '2 a.m.': 1,
 '9 degrees': 1,
 '85 degrees': 1,
 'Jan. 3': 1,
 '$3.2 billion': 1,
 '$3.6 billion': 1,
 '$1 , 600 ,': 1,
 'April 4 , 1968': 1,
 '2 million': 1,
 '8 million': 1,
 '1980': 1,
 'Poles': 1,
 '14 , 800 feet': 1,
 '$5 million': 1,
 '1985': 1,
 'Mexican': 1,
 '50 mph': 1,
 '10 percent': 2,
 '$60.6 million': 1,
 '3 p.m.': 1,
 'July 31 , 1989': 1,
 '1942.': 1,
 '1942': 1,
 '15 miles': 1,
 '6 million acres': 1,
 '1988': 1,
 '15 percent': 1,
 '$100 million': 1,
 '2 a.m. EDT': 1,
 '3 a.m. EDT': 1,
 '60 , 000 acres': 1,
 '40 million': 1,
 'March 16 , 1985.': 1,
 '2 p.m. EDT': 1,
 '7 -': 1,
 '84 percent': 1,
 '11 , 700 acres': 1,
 '28 percent': 1,
 'March 1994': 1,
 '1924 GMT': 1,
 '21 Mar 94': 1,
 '100 kilometres': 1,
 '6 Feb 94': 1,
 '1646 GMT': 1,
 '

In [11]:
entity_labels_count_true

{'Org': 198, 'Other': 133, 'Loc': 427, 'Peop': 321}

In [12]:
pip install openai


  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 21.1 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (352 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI
import json
import time
from tqdm import tqdm
import pandas as pd  # make sure df is defined with 'Content' column

# Set OpenAI API key
client = OpenAI(api_key = "")  # Replace with your actual API key

# Function: extract named_entities given allowed entity labels
def extract_named_entities_by_labels(paragraph, allowed_labels):
    allowed_labels_str = ", ".join(allowed_labels)

    system_prompt = (
        "You are a named entity recognition (NER) assistant. "
        "Your task is to extract named entities from a given paragraph, "
        "but only include entities whose type is one of the following: "
        f"{allowed_labels_str}. "
        "Return ONLY a JSON list of entity strings that match the allowed labels. "
        "Do not include the labels in the output. Keep it JSON parsable."
    )

    user_prompt = f"""
Paragraph:

{paragraph}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-4o-mini"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
        )

        reply = response.choices[0].message.content.strip()

        if reply.startswith("```json"):
            reply = reply[len("```json"):].strip()
        if reply.endswith("```"):
            reply = reply[:-len("```")].strip()

        return json.loads(reply)

    except Exception as e:
        print("Error with paragraph:", paragraph)
        print("Exception:", e)
        return []

    # Example: allowed labels
allowed_entity_labels = ["Person", "Organization", "Location", "Other"]

# Loop through DataFrame and extract entities
gpt_named_entities = []

for i in tqdm(range(len(test))):
    paragraph = ' '.join(test[i]['tokens'])
    entities = extract_named_entities_by_labels(paragraph, allowed_entity_labels)
    gpt_named_entities.append(entities)

  0%|          | 0/288 [00:00<?, ?it/s]

100%|██████████| 288/288 [03:27<00:00,  1.39it/s]


In [71]:
with open("conll04_gpt_named_entities_output.json", "w") as f:
    json.dump(gpt_named_entities, f, indent=2)

In [72]:
true=[]
for i in range(len(test)):
    temp=[]
    for j in test[i]['entities']:
        text=' '.join(test[i]['tokens'][j['start']:j['end']])
        temp.append(text)
    true.append(temp)

In [73]:
gpt_named_entities[34]

['President Nixon', 'Bush']

In [74]:
true[34]

['Watergate', 'U.S.', 'Bush']

In [75]:
their=0
ours=0
common=0
for i in range(len(gpt_named_entities)):
    their+=len(set(true[i]))
    ours+=len(set(gpt_named_entities[i]))
    common+=len(set(true[i]).intersection(set(gpt_named_entities[i])))

In [76]:
their,ours,common

(1059, 878, 692)

In [77]:
recall_avg=[]
precision_avg=[]
common2=0
for i in range(len(gpt_named_entities)):
    true_set=set(true[i])
    pred_set=set(gpt_named_entities[i])
    
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                
                if m in n or m[:-2]==n:
                    count+= 1
        recall = count / len(true_set) if len(true_set) > 0 else 0
        precision = count / len(pred_set) if len(pred_set) > 0 else 0
        common2 += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [78]:
common2

746

In [79]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.721516754850088, 0.7958829365079366)

In [ ]:
from openai import OpenAI
import json
import re

# Initialize OpenAI client
client = OpenAI(api_key = "")  # Replace with your actual API key

def extract_relation_labels_with_gpt_entities(paragraph, entities, relation_labels):
    """
    Extract RDF triples using GPT-4o/mini with entity spans.

    Args:
        paragraph (str): Full input paragraph
        entities (list): Each entity as [start, end, entity_type, entity_text]
        relation_labels (list): Allowed relation labels

    Returns:
        dict: {"relation_triples": [[head, relation, tail], ...]}
    """
    # Convert entity structure into readable text
    entity_descs = [
        f"[{start}, {end}, {etype}, {text}]"
        for start, end, etype, text in entities
    ]
    entity_str = "\n".join(entity_descs)
    relation_str = ", ".join(relation_labels)

    system_prompt = (
        "You are an expert in information extraction. "
        "Given a paragraph, a list of named entities with character spans and types, and a list of allowed relation labels, "
        "extract RDF relation triples in the format [head, relation, tail].\n\n"
        "- Head and tail must be from the provided entity list.\n"
        "- The relation must be from the relation_labels list.\n"
        "- The output must be ONLY a JSON object like:\n"
        '{ "relation_triples": [ ["Entity1", "Relation", "Entity2"], ... ] }\n'
        "- Do NOT include any explanation or extra text."
    )

    user_prompt = f"""
Paragraph:
\"\"\"
{paragraph}
\"\"\"

Entities (format: [start, end, type, entity]):
{entity_str}

Relation labels:
{relation_str}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # or "gpt-4o-mini"
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
        )

        reply = response.choices[0].message.content.strip()

        # Extract JSON block
        json_match = re.search(r'\{.*\}', reply, re.DOTALL)
        if json_match:
            json_text = json_match.group(0).strip()
            return json.loads(json_text)
        else:
            print("No JSON found.")
            print("Reply:", reply)
            return {"relation_triples": []}

    except Exception as e:
        print("Exception:", e)
        return {"relation_triples": []}
gpt_relation_triples = []
relation_labels = ['Located_In', 'Live_In', 'Work_For', 'Kill', 'OrgBased_In']

for i in tqdm(range(len(test))):
    paragraph = ' '.join(test[i]['tokens'])
    entities = ner[i]
    output= extract_relation_labels_with_gpt_entities(paragraph, entities, relation_labels)
    gpt_relation_triples.append(output['relation_triples'])

  0%|          | 0/288 [00:00<?, ?it/s]

100%|██████████| 288/288 [04:33<00:00,  1.05it/s]


In [81]:
with open("conll04_gpt_relation_triples_output.json", "w") as f:
    json.dump(gpt_relation_triples, f, indent=2)

In [82]:
relations_true=[]
for i in range(len(test)):
    temp=[]
    for j in test[i]['relations']:
        head=ner[i][j['head']][3]
        tail=ner[i][j['tail']][3]
        temp.append([head, tail, j['type']])
    relations_true.append(temp)

In [83]:
gpt_relation_triples[2]

[]

In [84]:
relations_true[2]

[['Reagan', 'America', 'Live_In']]

In [85]:
' '.join(test[7]['tokens'])

"Anastas Mikoyan , then Soviet first deputy premier , was the catalyst for the reversal of Khrushchev 's order , according to the authors ."

In [86]:
relations_true[7]

[['Anastas Mikoyan', 'Soviet', 'Live_In'], ['Khrushchev', 'Soviet', 'Live_In']]

In [87]:
recall_avg=[]
precision_avg=[]
common=0
ours=0
relations_all_filtered=[]
for i in range(len(relations_true)):
    true_set=relations_true[i]
    pred_set=gpt_relation_triples[i]
    d={}
    temp=[]
    for item in pred_set:
        if (item[0],item[2]) not in d:
            d[(item[0],item[2])]=item[1]
            temp.append([item[0],item[1],item[2]])
    relations_all_filtered.append(temp)
    
    ours+=len(d)
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if m[0]==n[0] and m[1]==n[2] and (m[2]==n[1]):
                    count+= 1
        recall = count/ len(true_set) if len(true_set) > 0 else 0
        precision = count / len(pred_set) if len(pred_set) > 0 else 0
        common += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [88]:
relations_true[0]

[['Hakawati Theatre', 'Jerusalem', 'OrgBased_In']]

In [89]:
gpt_relation_triples[0]

[['Hakawati Theatre', 'Located_In', 'Jerusalem'],
 ['Palestinians', 'Kill', 'Palestinians']]

In [90]:
sum(recall_avg) / len(recall_avg), sum(precision_avg) / len(precision_avg)

(0.4982931998556999, 0.4109760802469135)

In [65]:
their,ours,common

(1059, 297, 199)